In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%pip install -r "/content/drive/MyDrive/Deep learning models/requirements.txt"
%pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [4]:
import torch
import ast
import torch.nn as nn
import pandas as pd
import numpy as np
import torch.nn.functional as F

from Bio import SeqIO
from glob import glob
from transformers import (AutoModelForTokenClassification, AutoTokenizer,
                          AutoModelForMaskedLM, DataCollatorForTokenClassification,
                           EsmForMaskedLM, EsmTokenizer,
                           TrainingArguments, Trainer
                        )
from transformers.trainer_callback import ProgressCallback
from sklearn.metrics import (accuracy_score, precision_recall_fscore_support,
                             matthews_corrcoef, roc_auc_score)
from sklearn.model_selection import train_test_split
from pprint import pprint
from datasets import Dataset
from datetime import datetime

In [5]:
def convert_to_binary_list(original_binding_sites_lst, sequence_len):
    """Convert a Binding-Active site string to a binary list based on the sequence length."""
    binary_list = [0] * sequence_len  # Initialize a list of zeros

    # Ensure original_binding_sites_lst is a list and not empty
    if isinstance(original_binding_sites_lst, list) and len(original_binding_sites_lst) > 0:
        for idx in original_binding_sites_lst:
            if isinstance(idx, int) and 1 <= idx <= sequence_len:  # Ensure index is valid
                binary_list[idx - 1] = 1

    return binary_list

In [6]:
# binding_sites_df = pd.read_csv("/content/drive/MyDrive/Deep learning models/data/development_set/all_binding_sites_complete.csv")
# binding_sites_df['binding_sites'] = binding_sites_df['binding_sites'].apply(ast.literal_eval)
# # New concated binding sites df
# display(binding_sites_df)

In [7]:
# needed_binding_sites_df = binding_sites_df[binding_sites_df['sequence_length'] > 0]
# needed_binding_sites_df['binary_binding_sites'] = [
#             convert_to_binary_list(row['binding_sites'], row['sequence_length'])
#             for _, row in needed_binding_sites_df.iterrows()
#         ]

In [8]:
needed_training_sites_df = pd.read_csv("/content/drive/MyDrive/Deep learning models/data/development_set/official_de_dup_training_sites_df.csv")
needed_training_sites_df['binding_sites'] = needed_training_sites_df['binding_sites'].apply(ast.literal_eval)
needed_training_sites_df['binary_binding_sites'] = needed_training_sites_df['binary_binding_sites'].apply(ast.literal_eval)

display(needed_training_sites_df)

,prot_id,binding_sites,ligand_type,sequence,sequence_length,binary_binding_sites
0,P16113,"[31, 42, 46, 50, 52, 62, 66, 67, 68, 69, 70, 8...",small,MEHVAFGSEDIENTLAKMDDGQLDGLAFGAIQLDGDGNILQYNAAE...,125,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,O76242,"[11, 12, 22, 25, 26, 45, 48, 49, 52, 53, 66, 6...",small,MVNWAAVVDDFYQELFKAHPEYQNKFGFKGVALGSLKGNAAYKTQA...,110,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
2,P26789,"[1, 2, 3, 6, 7, 10, 15, 17, 18, 19, 20, 22, 23...",small,MNQGKIWTVVNPAIGIPALLGSVTVIAILVHLAILSHTTWFPAYWQ...,53,"[1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, ..."
3,P26790,"[33, 37, 39, 15, 16, 18, 19, 20, 21, 22, 23, 2...",small,ATLTAEQSEELHKYVIDGTRVFLGLALVAHFLAFSATPWLH,41,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
4,P21149,"[45, 46, 48, 50, 51, 52, 53, 83, 85, 55, 31]",small,MLSQVCRFGTITAVKGGVKKQLKFEDDQTLFTVLTEAGLMSADDTC...,100,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
1009,Q6N5V5,"[56, 59, 60, 63]",small,MTGPKQQPLPPDVEGREDAIEVLRAFVLDGGLSIAFMRAFEDPEMW...,98,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1010,C4LSE7,"[7, 8, 9, 10, 11, 44, 45, 12, 13, 123, 125, 126]",small,MKLLFVCLGNICRSPAAEAVMKKVIQNHHLTEKYICDSAGTCSYHE...,157,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, ..."
1011,Q0B0G9,"[143, 144, 145, 30, 44, 46, 47, 48, 49, 50, 62...",small,MLESVRKEWLEIMDRELLEKARSLINANYISTTLSTVDRNYEVNIA...,147,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1012,Q5SHE1,"[129, 130, 74, 107, 75, 14, 111, 48, 49, 47, 126]",small,MDLTHFQDGRPRMVDVTEKPETFRTATAEAFVELTEEALSALEKGG...,157,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."


In [9]:
np.random.seed(42)

train_df, val_df = train_test_split(
    needed_training_sites_df,
    test_size=0.1,  # 90% train, 10% test
    stratify=needed_training_sites_df['ligand_type'],
    random_state=42
)

# Verify the split ratios
print("Training set ligand distribution:")
print(train_df['ligand_type'].value_counts())
print("\nTest set ligand distribution:")
print(val_df['ligand_type'].value_counts())

# Calculate percentages to verify similar ratios
print("\nTraining set percentages:")
print(train_df['ligand_type'].value_counts(normalize=True))
print("\nTest set percentages:")
print(val_df['ligand_type'].value_counts(normalize=True))

Training set ligand distribution:
ligand_type
small      474
metal      344
nuclear     94
Name: count, dtype: int64

Test set ligand distribution:
ligand_type
small      53
metal      39
nuclear    10
Name: count, dtype: int64

Training set percentages:
ligand_type
small      0.519737
metal      0.377193
nuclear    0.103070
Name: proportion, dtype: float64

Test set percentages:
ligand_type
small      0.519608
metal      0.382353
nuclear    0.098039
Name: proportion, dtype: float64


In [10]:
test_df = pd.read_csv("/content/drive/MyDrive/Deep learning models/data/development_set/official_de_dup_testing_sites_df.csv")
test_df['binding_sites'] = test_df['binding_sites'].apply(ast.literal_eval)
test_df['binary_binding_sites'] = test_df['binary_binding_sites'].apply(ast.literal_eval)

test_df.sequence_length.describe()

,sequence_length
count,300.000000
mean,208.963333
std,105.970441
min,34.000000
25%,141.000000
50%,184.500000
75%,251.500000
max,813.000000


In [11]:
display(test_df.head())
display(train_df.head())

,prot_id,binding_sites,ligand_type,sequence,sequence_length,binary_binding_sites
0,P46859,"[160, 159, 163, 40, 17, 18, 19, 20, 21, 22, 23...",small,MSTTNHDHHIYVLMGVSGSGKSAVASEVAHQLHAAFLDGDFLHPRR...,175,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Q9HAN9,"[269, 14, 15, 16, 17, 23, 24, 27, 155, 156, 15...",small,MENSEKTEVVLLACGSFNPITNMHLRLFELAKDYMNGTGRYTVVKG...,279,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
2,P62799,"[64, 103, 85, 89, 60, 61]",small,MSGRGKGGKGLGKGGAKRHRKVLRDNIQGITKPAIRRLARRGGVKR...,103,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,P06897,"[65, 66, 37, 38, 40, 17, 18, 93, 58, 61, 62]",small,MSGRGKQGGKTRAKAKTRSSRAGLQFPVGRVHRLLRKGNYAERVGA...,130,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,P02281,"[35, 72, 106, 107, 109, 110, 83, 84]",small,MPEPAKSAPAPKKGSKKAVTKTQKKDGKKRRKSRKESYAIYVYKVL...,126,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


,prot_id,binding_sites,ligand_type,sequence,sequence_length,binary_binding_sites
473,Q8NI22,"[129, 131, 133, 135, 140, 81, 83, 85, 87, 92]",metal,MTMRSLLRTPFLCGLLWAFCAPGARAEEPAASFSQPGSMGLDKNTV...,146,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
390,P22452,"[2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 14, 17, 18, ...",nuclear,MGKKSKATKKRLAKLDNQNSRVPAWVMLKTDREVQRNHKRRHWRRN...,50,"[0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, ..."
19,P46926,"[1, 2, 262, 137, 138, 139, 141, 143, 145, 146,...",small,MKLIILEHYSQASEWAAKYIRNRIIQFNPGPEKYFTLGLPTGSTPL...,289,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
692,P74903,"[65, 67, 28, 22]",metal,MAVIADPETAQGFRLAGLEGYGASSAEEAQSLLETLVERGGYALVA...,104,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,P45718,"[137, 32, 39, 42, 43, 45, 46, 59, 62, 63, 66, ...",small,SLSDKDKAAVRALWSKIGKSSDAIGNDALSRMIVVYPQTKIYFSHW...,142,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


### Split sequences into chunks

In [12]:
def split_into_chunks(sequences, labels, chunk_size = 1000):
    """Split sequences and labels into chunks of size "chunk_size" or less."""
    new_sequences = []
    new_labels = []
    for seq, lbl in zip(sequences, labels):
        if len(seq) > chunk_size:
            # Split the sequence and labels into chunks of size "chunk_size" or less
            for i in range(0, len(seq), chunk_size):
                new_sequences.append(seq[i:i+chunk_size])
                new_labels.append(lbl[i:i+chunk_size])
        else:
            new_sequences.append(seq)
            new_labels.append(lbl)

    return new_sequences, new_labels

### Prepare training and testing sequences, labels

In [13]:
# Initial sequences
test_seq = test_df['sequence'].tolist()
test_labels = test_df['binary_binding_sites'].tolist()

train_seq = train_df['sequence'].tolist()
train_labels = train_df['binary_binding_sites'].tolist()

val_seq = val_df['sequence'].tolist()
val_labels = val_df['binary_binding_sites'].tolist()

In [14]:
# Apply new sequences by chunking
chunk_size = 1000

test_seq, test_labels = split_into_chunks(test_seq, test_labels, chunk_size)
train_seq, train_labels = split_into_chunks(train_seq, train_labels, chunk_size)
val_seq, val_labels = split_into_chunks(val_seq, val_labels, chunk_size)

In [15]:
print(len(train_seq[100]))
print(len(train_labels[100]))
print(train_labels[100])

106
106
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1]


### Tokenization and helper functions

In [16]:
pretrained_model = "facebook/esm2_t33_650M_UR50D"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
max_sequence_length = 1000 # here note that data was processed with chunks (context window) of 1000 residues - adapt accordingly

train_tokenized = tokenizer(train_seq, padding=True, truncation=True, max_length=max_sequence_length, return_tensors="pt", is_split_into_words=False)
eval_tokenized = tokenizer(val_seq, padding=True, truncation=True, max_length=max_sequence_length, return_tensors="pt", is_split_into_words=False)
test_tokenized = tokenizer(test_seq, padding=True, truncation=True, max_length=max_sequence_length, return_tensors="pt", is_split_into_words=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [17]:
train_tokenized

{'input_ids': tensor([[ 0, 20, 11,  ...,  1,  1,  1],
        [ 0, 20,  6,  ...,  1,  1,  1],
        [ 0, 20, 15,  ...,  1,  1,  1],
        ...,
        [ 0, 20, 14,  ...,  1,  1,  1],
        [ 0, 20,  6,  ...,  1,  1,  1],
        [ 0, 20, 13,  ...,  1,  1,  1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [18]:
def truncate_labels(labels, max_length):
    """Truncate labels to the specified max_length."""
    return [label[:max_length] for label in labels]

def compute_metrics_train(p):
    """Compute metrics for evaluation."""
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove padding (-100 labels)
    predictions = predictions[labels != -100].flatten()
    labels = labels[labels != -100].flatten()

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute precision, recall, F1 score, and AUC
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    auc = roc_auc_score(labels, predictions)

    # Compute MCC
    mcc = matthews_corrcoef(labels, predictions)

    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1, 'auc': auc, 'mcc': mcc}

def compute_loss(model, inputs):
    """Custom compute_loss function."""
    logits = model(**inputs).logits
    labels = inputs["labels"]
    loss_fct = nn.CrossEntropyLoss()
    active_loss = inputs["attention_mask"].view(-1) == 1
    active_logits = logits.view(-1, model.config.num_labels)
    active_labels = torch.where(
        active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels)
    )
    loss = loss_fct(active_logits, active_labels)
    return loss

In [19]:
# Verify some encoded and dedcode input, output
idx = 100
print(len(train_seq[idx]))
print(train_seq[idx])
encoded_input = tokenizer(train_seq[idx])
print(encoded_input)
decoded_output = tokenizer.decode(encoded_input['input_ids'])
print(decoded_output)

print(len(train_seq[idx]), len(encoded_input["input_ids"]))

106
MAQKKPKRNLSALKRHRQSLKRRLRNKAKKSAIKTLSKKAVQLAQEGKAEEALKIMRKAESLIDKAAKGSTLHKNAAARRKSRLMRKVRQLLEAAGAPLIGGGLSA
{'input_ids': [0, 20, 5, 16, 15, 15, 14, 15, 10, 17, 4, 8, 5, 4, 15, 10, 21, 10, 16, 8, 4, 15, 10, 10, 4, 10, 17, 15, 5, 15, 15, 8, 5, 12, 15, 11, 4, 8, 15, 15, 5, 7, 16, 4, 5, 16, 9, 6, 15, 5, 9, 9, 5, 4, 15, 12, 20, 10, 15, 5, 9, 8, 4, 12, 13, 15, 5, 5, 15, 6, 8, 11, 4, 21, 15, 17, 5, 5, 5, 10, 10, 15, 8, 10, 4, 20, 10, 15, 7, 10, 16, 4, 4, 9, 5, 5, 6, 5, 14, 4, 12, 6, 6, 6, 4, 8, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
<cls> M A Q K K P K R N L S A L K R H R Q S L K R R L R N K A K K S A I K T L S K K A V Q L A Q E G K A E E A L K I M R K A E S L I D K A A K G S T L H 

### Training

In [20]:
train_labels = truncate_labels(train_labels, max_sequence_length)
eval_labels = truncate_labels(val_labels, max_sequence_length)
test_labels = truncate_labels(test_labels, max_sequence_length)

train_dataset = Dataset.from_dict({k: v for k, v in train_tokenized.items()}).add_column("labels", train_labels)
eval_dataset = Dataset.from_dict({k: v for k, v in eval_tokenized.items()}).add_column("labels", eval_labels)
test_dataset = Dataset.from_dict({k: v for k, v in test_tokenized.items()}).add_column("labels", test_labels)

In [21]:
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

model = AutoModelForTokenClassification.from_pretrained(pretrained_model, num_labels = 2)
training_args = TrainingArguments(
    output_dir=f"/content/drive/MyDrive/Deep learning models/data/trained_models/esm2_t33_650M-binding-sites_{timestamp}",
    seed = 42,
    num_train_epochs = 10,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
    metric_for_best_model = "eval_mcc",
    greater_is_better = True,
    logging_dir = f"/content/drive/MyDrive/Deep learning models/data/trained_models/esm2_t33_650M-binding-sites_{timestamp}",
    logging_strategy = "steps",
    logging_steps = 10,
    save_total_limit = 2,
    bf16 = True,
    report_to= "none" # Disable Wandb
)

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.61G [00:00<?, ?B/s]

Some weights of EsmForTokenClassification were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Customized model implementation

In [22]:
# Add custom CNN layer

class CNNLayer(nn.Module):
    def __init__(self, in_channels=2, out_channels=256, kernel_sizes=[3, 5, 7]):  # Changed in_channels default to 2
        super().__init__()
        self.input_projection = nn.Linear(in_channels, 768)  # Add projection layer
        self.convs = nn.ModuleList([
            nn.Conv1d(768, out_channels, kernel_size, padding=kernel_size//2)
            for kernel_size in kernel_sizes
        ])
        self.norm = nn.LayerNorm(out_channels * len(kernel_sizes))

    def forward(self, x):
        # x shape: (batch_size, seq_len, in_channels)
        if len(x.shape) == 3 and x.shape[1] == 2:
            x = x.transpose(1, 2)

        x = self.input_projection(x)
        x = x.transpose(1, 2)  # (batch_size, 768, seq_len)

        # Apply convolutions
        conv_outputs = []
        for conv in self.convs:
            conv_out = F.relu(conv(x))
            conv_outputs.append(conv_out)

        x = torch.cat(conv_outputs, dim=1)
        x = x.transpose(1, 2)

        return self.norm(x)

class CNNEnhancedPositionAwareLoss(nn.Module):
    def __init__(self, pos_weight=8.0, position_weight=1.5):
        super().__init__()
        self.weighted_ce = WeightedCrossEntropyLoss(pos_weight)
        self.position_weight = position_weight

        # Add CNN layers for feature extraction
        self.cnn = CNNLayer(in_channels = 2, out_channels = 256)  # Adjust dimensions as needed
        self.output_proj = nn.Linear(768, 2)  # Project back to original output dimension

    def forward(self, logits, labels, attention_mask):
        # Apply CNN processing
        enhanced_features = self.cnn(logits)
        enhanced_logits = self.output_proj(enhanced_features)

        # Base loss computation
        base_loss = self.weighted_ce(enhanced_logits, labels, attention_mask)

        # Position-aware component with CNN features
        probs = torch.softmax(enhanced_logits, dim=-1)[:,:,1]
        position_loss = torch.tensor(0.0).to(logits.device)

        batch_size = logits.shape[0]
        for b in range(batch_size):
            for i in range(1, logits.shape[1]-1):
                if labels[b,i] == 1 or labels[b,i-1] == 1 or labels[b,i+1] == 1:
                    position_loss += torch.abs(probs[b,i] - (labels[b,i] == 1).float())

        return base_loss + self.position_weight * position_loss

In [23]:
# trainer = Trainer(
#     model = model,
#     args = training_args,
#     train_dataset = train_dataset,
#     eval_dataset = eval_dataset,
#     compute_metrics = compute_metrics_train,
#     tokenizer = tokenizer,
#     data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer),
#     callbacks = [
#         ProgressCallback()
#     ]
# )

class WeightedCrossEntropyLoss(nn.Module):
    def __init__(self, pos_weight=8.0):  # High weight since binding sites are rare, edit soon for experiments
        super().__init__()
        self.pos_weight = pos_weight

    def forward(self, logits, labels, attention_mask):
        # Handle class imbalance with weighted loss
        weight = torch.ones(2).to(logits.device)
        weight[1] = self.pos_weight  # Higher weight for binding sites

        loss_fct = nn.CrossEntropyLoss(weight=weight, ignore_index=-100)
        active_loss = attention_mask.view(-1) == 1
        active_logits = logits.view(-1, logits.shape[-1])
        active_labels = torch.where(active_loss, labels.view(-1),
                                  torch.tensor(loss_fct.ignore_index).type_as(labels))

        return loss_fct(active_logits, active_labels)


# Add position-aware loss
class PositionAwareLoss(nn.Module):
    def __init__(self, pos_weight=8.0, position_weight=1.5):
        super().__init__()
        self.weighted_ce = WeightedCrossEntropyLoss(pos_weight)
        self.position_weight = position_weight

    def forward(self, logits, labels, attention_mask):
        base_loss = self.weighted_ce(logits, labels, attention_mask)

        # Position-aware component
        probs = torch.softmax(logits, dim=-1)[:,:,1]  # Get binding probabilities
        position_loss = torch.tensor(0.0).to(logits.device)

        # Penalize offset predictions
        batch_size = logits.shape[0]
        for b in range(batch_size):
            for i in range(1, logits.shape[1]-1):
                # Encourage predictions to match true binding site positions
                if labels[b,i] == 1 or labels[b,i-1] == 1 or labels[b,i+1] == 1:
                    position_loss += torch.abs(probs[b,i] - (labels[b,i] == 1).float())

        return base_loss + self.position_weight * position_loss


class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_fct = PositionAwareLoss(pos_weight=8.0, position_weight=0.5)

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        logits = model(**inputs).logits
        labels = inputs["labels"]
        attention_mask = inputs["attention_mask"]

        loss = self.loss_fct(logits, labels, attention_mask)

        return (loss, {"logits": logits}) if return_outputs else loss

# Create trainer with custom loss computation
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    compute_metrics = compute_metrics_train,
    tokenizer = tokenizer,
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer),
    callbacks = [ProgressCallback()]
)

<ipython-input-23-77f840cd27b9>:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


In [24]:
trainer.train()

  0%|          | 0/1140 [00:00<?, ?it/s]

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Auc,Mcc
1,36.558500,44.434532,0.676131,0.137211,0.609274,0.223980,0.645480,0.163391
2,20.291400,39.451084,0.689611,0.152224,0.666667,0.247855,0.679092,0.201820
3,11.902000,38.859585,0.602977,0.133577,0.761073,0.227266,0.675457,0.188277
4,6.813300,37.838345,0.726922,0.170044,0.659597,0.270383,0.696056,0.228253
5,11.965800,39.019829,0.808691,0.217876,0.576835,0.316288,0.702395,0.267778
6,8.412000,40.498409,0.850612,0.259958,0.512996,0.345059,0.695830,0.290814
7,3.811500,40.466778,0.854679,0.268838,0.520067,0.354450,0.701274,0.301359
8,4.879300,42.254494,0.880761,0.311297,0.457268,0.370420,0.686607,0.314116
9,4.941600,42.532494,0.883983,0.319196,0.452277,0.374258,0.686064,0.318181
10,2.643000,45.178608,0.901610,0.365685,0.384695,0.374949,0.664627,0.321710


{'loss': 52.5539, 'grad_norm': 24.474546432495117, 'learning_rate': 4.956140350877193e-05, 'epoch': 0.09}
{'loss': 49.9664, 'grad_norm': 45.11398696899414, 'learning_rate': 4.912280701754386e-05, 'epoch': 0.18}
{'loss': 44.6234, 'grad_norm': 20.68365478515625, 'learning_rate': 4.868421052631579e-05, 'epoch': 0.26}
{'loss': 51.5255, 'grad_norm': 100.30633544921875, 'learning_rate': 4.824561403508772e-05, 'epoch': 0.35}
{'loss': 55.7431, 'grad_norm': 28.646896362304688, 'learning_rate': 4.780701754385965e-05, 'epoch': 0.44}
{'loss': 44.5071, 'grad_norm': 30.01299285888672, 'learning_rate': 4.736842105263158e-05, 'epoch': 0.53}
{'loss': 45.9363, 'grad_norm': 44.973018646240234, 'learning_rate': 4.6929824561403515e-05, 'epoch': 0.61}
{'loss': 47.9145, 'grad_norm': 59.34453582763672, 'learning_rate': 4.649122807017544e-05, 'epoch': 0.7}
{'loss': 44.3488, 'grad_norm': 42.887149810791016, 'learning_rate': 4.605263157894737e-05, 'epoch': 0.79}
{'loss': 40.0733, 'grad_norm': 64.83621215820312, 

  0%|          | 0/38 [00:00<?, ?it/s]

{'eval_loss': 44.434532165527344, 'eval_accuracy': 0.6761313787107786, 'eval_precision': 0.13721082701133278, 'eval_recall': 0.6092742773965482, 'eval_f1': 0.22398043037877918, 'eval_auc': 0.6454802624024898, 'eval_mcc': 0.1633906238462479, 'eval_runtime': 32.8552, 'eval_samples_per_second': 9.131, 'eval_steps_per_second': 1.157, 'epoch': 1.0}
{'loss': 24.1124, 'grad_norm': 120.0439224243164, 'learning_rate': 4.473684210526316e-05, 'epoch': 1.05}
{'loss': 29.4833, 'grad_norm': 77.1359634399414, 'learning_rate': 4.429824561403509e-05, 'epoch': 1.14}
{'loss': 24.5508, 'grad_norm': 101.63155364990234, 'learning_rate': 4.3859649122807014e-05, 'epoch': 1.23}
{'loss': 24.9003, 'grad_norm': 63.653316497802734, 'learning_rate': 4.342105263157895e-05, 'epoch': 1.32}
{'loss': 24.9678, 'grad_norm': 84.70701599121094, 'learning_rate': 4.298245614035088e-05, 'epoch': 1.4}
{'loss': 18.3854, 'grad_norm': 376.3615417480469, 'learning_rate': 4.254385964912281e-05, 'epoch': 1.49}
{'loss': 18.9937, 'grad

...

  0%|          | 0/38 [00:00<?, ?it/s]

{'eval_loss': 42.532493591308594, 'eval_accuracy': 0.8839828359042257, 'eval_precision': 0.31919577340769006, 'eval_recall': 0.45227698066126015, 'eval_f1': 0.37425793684935044, 'eval_auc': 0.6860641986927586, 'eval_mcc': 0.3181812837154912, 'eval_runtime': 33.747, 'eval_samples_per_second': 8.89, 'eval_steps_per_second': 1.126, 'epoch': 9.0}
{'loss': 3.9993, 'grad_norm': 1.99802565574646, 'learning_rate': 4.824561403508772e-06, 'epoch': 9.04}
{'loss': 3.7218, 'grad_norm': 49.26662826538086, 'learning_rate': 4.3859649122807014e-06, 'epoch': 9.12}
{'loss': 4.0845, 'grad_norm': 40.5963134765625, 'learning_rate': 3.9473684210526315e-06, 'epoch': 9.21}
{'loss': 4.074, 'grad_norm': 1.8521372079849243, 'learning_rate': 3.5087719298245615e-06, 'epoch': 9.3}
{'loss': 4.2797, 'grad_norm': 79.73377227783203, 'learning_rate': 3.070175438596491e-06, 'epoch': 9.39}
{'loss': 4.0939, 'grad_norm': 2.4408340454101562, 'learning_rate': 2.631578947368421e-06, 'epoch': 9.47}
{'loss': 1.9976, 'grad_norm': 

  0%|          | 0/38 [00:00<?, ?it/s]

{'eval_loss': 45.17860794067383, 'eval_accuracy': 0.9016095327728948, 'eval_precision': 0.36568491796797786, 'eval_recall': 0.38469536286130174, 'eval_f1': 0.37494933117146334, 'eval_auc': 0.6646265342295451, 'eval_mcc': 0.3217102004731047, 'eval_runtime': 36.7553, 'eval_samples_per_second': 8.162, 'eval_steps_per_second': 1.034, 'epoch': 10.0}
{'train_runtime': 1629.1868, 'train_samples_per_second': 5.598, 'train_steps_per_second': 0.7, 'train_loss': 12.553287708550169, 'epoch': 10.0}


TrainOutput(global_step=1140, training_loss=12.553287708550169, metrics={'train_runtime': 1629.1868, 'train_samples_per_second': 5.598, 'train_steps_per_second': 0.7, 'total_flos': 1.982744217908928e+16, 'train_loss': 12.553287708550169, 'epoch': 10.0})

### End of customized model implementation

### Evaluation

In [27]:
from accelerate import Accelerator

accelerator = Accelerator()
saved_model_path = f"/content/drive/MyDrive/Deep learning models/data/trained_models/esm2_t33_650M-binding-sites_2025-02-20_11-38-46/checkpoint-1140"
loaded_model = AutoModelForTokenClassification.from_pretrained(saved_model_path)
model = accelerator.prepare(loaded_model)

In [28]:
# Define label mappings
id2label = {0: "No binding site", 1: "Binding site"}
label2id = {v: k for k, v in id2label.items()}

# Create a data collator
data_collator = DataCollatorForTokenClassification(tokenizer)

In [29]:
def compute_metrics_dataset(dataset, data_collator):
    # Get the predictions using the trained model
    trainer = Trainer(model=model, data_collator=data_collator)
    predictions, labels, _ = trainer.predict(test_dataset=dataset)

    # Remove padding and special tokens
    mask = labels != -100
    true_labels = labels[mask].flatten()
    print(f"True labels: {list(true_labels)}")
    flat_predictions = np.argmax(predictions, axis=2)[mask].flatten().tolist()
    print(f"Flat predictions: {flat_predictions}")

    # Compute the metrics
    accuracy = accuracy_score(true_labels, flat_predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, flat_predictions, average='binary')
    auc = roc_auc_score(true_labels, flat_predictions)
    mcc = matthews_corrcoef(true_labels, flat_predictions)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, "auc": auc, "mcc": mcc}

In [30]:
## Wandb API key: 8e2ece42312dcb7bef3cf33270f391abb4e5cf5c
test_metrics = compute_metrics_dataset(test_dataset, data_collator)
print(test_metrics)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dangkhoa20006 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


True labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### Inference

In [ ]:
model.eval()

In [ ]:
def infer_binding_sites(sequence, tokenizer, model, accelerator):
  tokenized_sequence = tokenizer(sequence, return_tensors="pt", truncation=True, max_length=1024, padding='max_length')

  tokenized_sequence = {key: value.to(accelerator.device) for key, value in tokenized_sequence.items()}
  tokens = tokenizer.convert_ids_to_tokens(tokenized_sequence["input_ids"][0])  # Convert input ids back to tokens (aminoacids in this case, but also special tokens generated by the tokenizer)

  with torch.no_grad():
      logits = model(**tokenized_sequence).logits

  predictions = torch.argmax(logits, dim=2)
  predictions = predictions[0].cpu().numpy()


  special_tokens = [tokenizer.pad_token, tokenizer.cls_token, tokenizer.sep_token, tokenizer.eos_token]
  special_tokens = [token for token in special_tokens if token is not None]  # Remove None tokens
  filtered_predictions = []
  for token, pred in zip(tokens, predictions):
    if token not in special_tokens:
      filtered_predictions.append(pred)

  if len(filtered_predictions) != len(sequence):
      error_message = (
          f"Error: Length mismatch! Sequence length: {len(sequence)}, "
          f"Filtered predictions length: {len(filtered_predictions)}. "
          "This could be due to unaccounted special tokens in the tokenizer."
      )
      raise ValueError(error_message)
  return filtered_predictions

In [ ]:
sample_idx = 100
sample_sequence = test_df.iloc[sample_idx]['sequence']
print(f"Sequence: {sample_sequence}")

filtered_predictions = infer_binding_sites(sample_sequence, tokenizer, model, accelerator)
print(f"Filtered predictions: {filtered_predictions}")